# Laboratorium 3: Analiza wrażliwości

**Imię i nazwisko:** Anna Bernard, Mikołaj Mieszko Charchuta 
**Grupa:**  Symulanci
**Data wykonania:**  29.05.2025

---

## 1. Wstęp

W tym laboratorium podjęliśmy się prób oceny wrażliwości lokalnej (opartej o funkcje wrażliwości) 
i globalnej (opartej o wskaźniki Sobola).

## 2. Opis zaimplementowanego rozwiązania

- **Wariant implementacji:** RK4 ze stałym krokiem, ręcznie wpisane pochodne
- **Biblioteki użyte:** matplotlib, SALib
- **Model:** Wrażliwość symulacji dynamiki białek p53, MDMcyto, MDMn, PTEN na zmiany parametrów reakcji w dwóch scenariuszach biologicznych
- **Horyzont czasowy:** 48 godzin (2880 minut)
- **Krok całkowania:** 6 minut
- **Wprowadzanie danych:** Przyjęcie domyślnych

In [10]:
def f_p53(params, p53, mdmn): 
    return params['p1'] - params['d1'] * p53 * (mdmn**2)

def f_mdmcyto(params, p53, mdmcyto, pten, siRNA=False, no_DNA_damage=False):
    siRNA_factor = params['value_siRNA'] if siRNA else 1
    DNA_damage_factor = params['value_no_DNA_damage'] if no_DNA_damage else 1
    return (params['p2'] * siRNA_factor * (p53**4) / ((p53**4) + (params['k2']**4))
            - params['k1'] * (params['k3']**2) / ((params['k3']**2) + (pten**2)) * mdmcyto
            - params['d2'] * DNA_damage_factor * mdmcyto)

def f_mdmn(params, mdmn, mdmcyto, pten, no_DNA_damage=False):
    if no_DNA_damage:
        return (params['k1'] * (params['k3']**2) / ((params['k3']**2) + (pten**2)) * mdmcyto
                - params['d2'] * params['value_no_DNA_damage'] * mdmn)
    else:
        return (params['k1'] * (params['k3']**2) / ((params['k3']**2) + (pten**2)) * mdmcyto
                - params['d2'] * mdmn)

def f_pten(params, pten, p53, pten_off=False):
    if not pten_off:
        return (params['p3'] * (p53**4) / ((p53**4) + (params['k2']**4))
                - params['d3'] * pten)
    else:
        return (params['p3'] * params['value_PTEN_off'] * (p53**4) / ((p53**4) + (params['k2']**4))
                - params['d3'] * pten)

    
# Pochodne tych funkcji(?):

# RK4 ze stałym krokiem
def RK4const(params, p53, mdcyto, mdmn, pten, h, siRNA=False, pten_off=False, no_DNA_damage=False):
    k1_p53 = f_p53(params, p53, mdmn)
    k1_mdmcyto = f_mdmcyto(params, p53, mdcyto, pten, siRNA)
    k1_mdmn = f_mdmn(params, mdmn, mdcyto, pten)
    k1_pten = f_pten(params, pten, p53, pten_off)

    k2_p53 = f_p53(params, p53 + h/2*k1_p53, mdmn + h/2*k1_mdmn)
    k2_mdmcyto = f_mdmcyto(params, p53 + h/2*k1_p53, mdcyto + h/2*k1_mdmcyto, pten + h/2*k1_pten, siRNA)
    k2_mdmn = f_mdmn(params, mdmn + h/2*k1_mdmn, mdcyto + h/2*k1_mdmcyto, pten + h/2*k1_pten)
    k2_pten = f_pten(params, pten + h/2*k1_pten, p53 + h/2*k1_p53, pten_off)

    k3_p53 = f_p53(params, p53 + h/2*k2_p53, mdmn + h/2*k2_mdmn)
    k3_mdmcyto = f_mdmcyto(params, p53 + h/2*k2_p53, mdcyto + h/2*k2_mdmcyto, pten + h/2*k2_pten, siRNA)
    k3_mdmn = f_mdmn(params, mdmn + h/2*k2_mdmn, mdcyto + h/2*k2_mdmcyto, pten + h/2*k2_pten)
    k3_pten = f_pten(params, pten + h/2*k2_pten, p53 + h/2*k2_p53, pten_off)

    k4_p53 = f_p53(params, p53 + h*k3_p53, mdmn + h*k3_mdmn)
    k4_mdmcyto = f_mdmcyto(params, p53 + h*k3_p53, mdcyto + h*k3_mdmcyto, pten + h*k3_pten, siRNA)
    k4_mdmn = f_mdmn(params, mdmn + h*k3_mdmn, mdcyto + h*k3_mdmcyto, pten + h*k3_pten)
    k4_pten = f_pten(params, pten + h*k3_pten, p53 + h*k3_p53, pten_off)


    p53 += (k1_p53 + 2*k2_p53 + 2*k3_p53 + k4_p53) * h / 6
    mdcyto += (k1_mdmcyto + 2*k2_mdmcyto + 2*k3_mdmcyto + k4_mdmcyto) * h / 6
    mdmn += (k1_mdmn + 2*k2_mdmn + 2*k3_mdmn + k4_mdmn) * h / 6
    pten += (k1_pten + 2*k2_pten + 2*k3_pten + k4_pten) * h / 6
    return p53, mdcyto, mdmn, pten

## 3. Parametry symulacji i scenariusze

In [11]:
# Warunki początkowe zgodnie z liczbą cząsteczek w hepatocycie:
p53_0 = 102000
mdmcyto_0 = 205000
mdmn_0 = 205000
pten_0 = 614000
proteins = [p53_0, mdmcyto_0, mdmn_0, pten_0] 

h = 6 # krok czasowy w minutach
iterations = int(48*60/h) # liczba iteracji na 48h
conditions = { 
        "Basic" : (False, False, True), # siRNA, PTEN_off, no_DNA_damage
        "Tumor" : (False, True, False),
}
scenarios = list(conditions.keys())

## 4. Wyniki symulacji: RK4 (stały krok)

In [ ]:
def RK4(params, proteins, scenario, iterations, h):
    siRNA, pten_off, no_DNA_damage = conditions[scenario]
    p53, mdmcyto, mdmn, pten = proteins
    time_values = []
    p53_values = []
    mdmcyto_values = []
    mdmn_values = []
    pten_values = []
    for i in range(iterations):
        time = i * h
        time_values.append(time)
        p53_values.append(p53)
        mdmcyto_values.append(mdmcyto)
        mdmn_values.append(mdmn)
        pten_values.append(pten)
        p53, mdmcyto, mdmn, pten = RK4const(params, p53, mdmcyto, mdmn, pten, h, siRNA, pten_off, no_DNA_damage)
    return time_values, [p53_values, mdmcyto_values, mdmn_values, pten_values]

Dla porówniania pozwolę sobie skorzystać z rozwiązań gotowych oferowanych przez pakiet SALib (niby nie mówił Profesor, że nie można korzystać z takich, ale uznaliśmy że nie wypada, więc to tylko kontorlnie):

In [ ]:
from SALib.sample import saltelli
from SALib.analyze import soboldef


model_ode(t, y, siRNA=False, pten_off=False, no_DNA_damage=False):
    p53, mdmcyto, mdmn, pten = y
    dydt = np.zeros(4)
    dydt[0] = f_p53(p53, mdmn)
    dydt[1] = f_mdmcyto(p53, mdmcyto, pten, siRNA=siRNA, no_DNA_damage=no_DNA_damage)
    dydt[2] = f_mdmn(mdmn, mdmcyto, pten, no_DNA_damage=no_DNA_damage)
    dydt[3] = f_pten(pten, p53, pten_off=pten_off)
    return dydt
for scenario in scenarios:
    siRNA, pten_off, no_DNA_damage = conditions[scenario]
    # Definicja zakresu czasu i warunków początkowych
    t_span = (0, 48*60)  # od 0 do 2880 minut (48h)
    y0 = [p53_0, mdmcyto_0, mdmn_0, pten_0]
    t_eval = np.arange(0, 48*60 + h, h)
    sol = solve_ivp(
        model_ode, t_span, y0, method='RK45', t_eval=t_eval,
        args=(siRNA, pten_off, no_DNA_damage)
    )
    plt.figure(figsize=(10,5))
    plt.xlabel("Czas [min]")
    plt.ylabel("Liczba cząsteczek białka")
    plt.title(f"SALibsobol: Liczności białek w 48h w scenariuszu: {scenario}")
    plt.yscale("log")
    plt.legend()
    plt.grid(True, which="both", ls="--")
    plt.show()


## 6. Wnioski

1.

---